# Import libraries and database connection

In [1]:
from pymongo import MongoClient
import json
import re
import csv
import pandas as pd
import json
from bson import ObjectId
import matplotlib.pyplot as plt
import seaborn as sns

# Create a MongoClient object and specify the MongoDB connection URL
url = "mongodb://localhost:27017/"
client1 = MongoClient(url)

# Access a specific database
db1 = client1["youtube_twitter_db1"]

# Send a ping to confirm a successful connection
try:
    client1.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


# Retrieving collections

In [2]:
tw_collection = db1.tw_collection
yt_collection = db1.yt_collection

# Associating to each tweet a label 1 if it refers to a moderated video

In [12]:
# Retrieve all tweet IDs linked to moderated videos
moderated_tweet_ids = []
for doc in yt_collection.find({'moderationStatus': 'moderated'}, {'tweet_ids': 1}):
    moderated_tweet_ids.extend(doc['tweet_ids'])


In [15]:
#remove duplicates
moderated_tweet_ids = list((set(moderated_tweet_ids)))

In [ ]:
# Iterate over documents in tw_collection
for doc in tw_collection.find():
    tweet_id = doc['tweetid']
    moderated = 1 if tweet_id in moderated_tweet_ids else 0

    # Update the document in tw_collection with the moderated field
    tw_collection.update_one(
        {'_id': doc['_id']},
        {'$set': {'moderated': moderated}}
    )

# Save collection as json

In [18]:
# Custom JSON encoder class to handle ObjectId serialization
class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return super().default(o)

# Retrieve all documents from the collection
all_documents = list(tw_collection.find())

# Specify the path and filename for the output JSON file
output_file = "tw_collection_w_status.json"

# Save the collection data to the JSON file
with open(output_file, "w") as file:
    json.dump(all_documents, file, cls=JSONEncoder)

print("Collection data saved as JSON.")


Collection data saved as JSON.


# Save as a csv

In [19]:
 # Retrieve documents from the collection
cursor = tw_collection.find()

# Convert documents to DataFrame
df = pd.DataFrame(list(cursor))

# Export DataFrame to CSV
df.to_csv("tw_collection_w_status.csv", index=False)